In [1]:
YOUNG_GENERATION = 0

class PyGCHead(object):
    def __init__(self, instance):
        self.refs = 1
        self.instance = instance

    def __repr__(self):
        return "PyGCHead refs: {} instance: {}".format(self.refs,
                                                       self.instance)

In [2]:
class GCObject(object):
    def __init__(self, rc=0):
        self.rc = rc

    def tp_traverse(self, fn):
        for k, v in self.__dict__.items():
            if k != 'rc' and v:
                fn(v)

In [3]:
class GCList(list, GCObject):
    def __init__(self):
        GCObject.__init__(self)

    def tp_traverse(self, fn):
        for elem in self:
            fn(elem)

In [4]:
class Foo(GCObject):
    def __init__(self, **kwargs):
        super(Foo, self).__init__(**kwargs)
        self.my_list = None
        self.bar = None

In [5]:
class Bar(GCObject):
    def __init__(self, **kwargs):
        super(Bar, self).__init__(**kwargs)
        self.foo = None

In [6]:
class GC(object):
    def __init__(self):
        self.generations = [([], 2),
                            ([], 3)]

    def allocate(self, kls, *args, **kwargs):
        instance = kls(*args, **kwargs)
        self.generations[YOUNG_GENERATION][0].append(PyGCHead(instance))
        if len(self.generations[0][0]) > self.generations[0][1]:
            self.collect(generation=0)

        return instance

    def search_pygc(self, instance):
        for e in self.generations:
            for a in e[0]:
                if a.instance == instance:
                    return a

    def update_refs(self, generation_list):
        for pygc in generation_list:
            pygc.refs = pygc.instance.rc

    def visit_reachable(self, instance):
        pygc = self.search_pygc(instance)
        # We thought that was unreachable but at the end is reachable
        if pygc.refs == 0:
            pygc.refs = 1
        else:
            assert pygc.refs > 0

    def visit_decref(self, instance):
        pygc = self.search_pygc(instance)
        # We're only interested in objects of this generation
        # older generations refs value is negative.
        if pygc.refs > 0:
            pygc.refs -= 1

    def substract_refs(self, generation_list):
        for pygc in generation_list:
            pygc.instance.tp_traverse(self.visit_decref)

    def move_unreachable(self, young):
        for pygc in young:
            # It's reachable from outside
            if pygc.refs > 0:
                pygc.instance.tp_traverse(self.visit_reachable)

        unreachable = [pygc for pygc in young if pygc.refs == 0]
        young = [pygc for pygc in young if pygc.refs > 0]
        return young, unreachable

    def move(self, young, old):
        old.extend(young)

    def delete(self, unreachable):
        for pygc in unreachable:
            print("Byeee cruel world! ", pygc)


    def collect(self, generation):
        young = self.generations[generation][0]
        old = self.generations[generation + 1][0]

        self.update_refs(young)
        self.substract_refs(young)

        unreachable = []
        young, unreachable = self.move_unreachable(young)

        self.move(young, old)

        # handle weakrefs, finalizers

        self.delete(unreachable)

    def reset_generations(self):
        self.generations = [([], 2),
                            ([], 3)]

    def run_code(self, fn):
        self.reset_generations()
        fn(self)

In [7]:
def example1(gc):
    import pdb; pdb.set_trace()
    l = gc.allocate(GCList)
    foo = gc.allocate(Foo)
    l.append(foo)
    foo.rc += 1
    foo.my_list = l
    l.rc += 1
    foo2 = gc.allocate(Foo, rc=2)
    foo2.rc += 1

In [ ]:
def example2(gc):
    import pdb; pdb.set_trace()
    foo = gc.allocate(Foo)
    bar = gc.allocate(Bar)
    bar.foo = foo
    foo.rc += 1
    foo.bar = bar
    bar.rc += 1
    foo2 = gc.allocate(Foo, rc=2)

In [ ]:
gc = GC()
gc.run_code(example1)

> <ipython-input-7-76fc184fbffd>(3)example1()
-> l = gc.allocate(GCList)
(Pdb) n
> <ipython-input-7-76fc184fbffd>(4)example1()
-> foo = gc.allocate(Foo)
(Pdb) n
> <ipython-input-7-76fc184fbffd>(5)example1()
-> l.append(foo)
(Pdb) 
> <ipython-input-7-76fc184fbffd>(6)example1()
-> foo.rc += 1
(Pdb) 
> <ipython-input-7-76fc184fbffd>(7)example1()
-> foo.my_list = l
(Pdb) 
> <ipython-input-7-76fc184fbffd>(8)example1()
-> l.rc += 1
(Pdb) 
> <ipython-input-7-76fc184fbffd>(9)example1()
-> foo2 = gc.allocate(Foo, rc=2)
(Pdb) 
Byeee cruel world!  PyGCHead refs: 0 instance: [<__main__.Foo object at 0x7f1a0ed918d0>]
Byeee cruel world!  PyGCHead refs: 0 instance: <__main__.Foo object at 0x7f1a0ed918d0>
> <ipython-input-7-76fc184fbffd>(10)example1()
-> foo2.rc += 1
(Pdb) 
--Return--
> <ipython-input-7-76fc184fbffd>(10)example1()->None
-> foo2.rc += 1
(Pdb) 
--Return--
> <ipython-input-6-93c1f951a9ad>(83)run_code()->None
-> fn(self)
(Pdb) 
--Call--
> /home/fran/.venvs/py3/lib/python3.4/site-packages/